# `quickmt` Client Demo

In [ ]:
from pathlib import Path

import datasets
from sacrebleu import BLEU, CHRF

from quickmt import Translator

In [ ]:
t = Translator("/path/to/model")

In [ ]:
t(["他补充道：“我们现在有 4 个月大没有糖尿病的老鼠，但它们曾经得过该病。”"], beam_size=1)

In [ ]:
bleu = BLEU()
chrf = CHRF()
src_lang = "zho_Hans"
tgt_lang = "eng_Latn"

try:
    flores = datasets.load_dataset(
        "facebook/flores",
        f"{src_lang}-{tgt_lang}",  # trust_remote_code=True
    )
except:
    flores = datasets.load_dataset(
        "facebook/flores",
        f"{tgt_lang}-{src_lang}",  # trust_remote_code=True
    )

src = []
ref = []
for i in flores["devtest"]:
    src.append(i[f"sentence_{src_lang}"])
    ref.append(i[f"sentence_{tgt_lang}"])

In [ ]:
results = t(src, beam_size=1)

In [ ]:
results[:5]

In [ ]:
print(bleu.corpus_score(results, [ref]))
print(chrf.corpus_score(results, [ref]))